In [1]:
"""
TODO:
* features to add: conf, SOS (?)
+ diffVec idea: https://adeshpande3.github.io/Applying-Machine-Learning-to-March-Madness
+ elo idea: https://towardsdatascience.com/machine-learning-madness-predicting-every-ncaa-tournament-matchup-7d9ce7d5fc6d
"""

# import modules
import pandas as pd
import numpy as np
from functools import reduce

## Preparations: Load & Manipulate Data

In [2]:
# load raw datasets
dfTeamNames = pd.read_csv("../data/raw/WTeams.csv")
dfSeasonResultsDet = pd.read_csv("../data/raw/WRegularSeasonDetailedResults.csv")

dfSeasonResultsDet.tail()

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  WFGM  \
46337    2018     132     3195      68     3239      58    H      0    24   
46338    2018     132     3251      60     3421      42    H      0    21   
46339    2018     132     3311      69     3372      65    N      0    24   
46340    2018     132     3343      63     3335      34    N      0    21   
46341    2018     132     3384      66     3352      56    H      0    19   

       WFGA  ...  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  
46337    58  ...     14     9    13   14   18     8   19     5     5   19  
46338    56  ...     11    12    15    8   22     4   10     3     1   14  
46339    47  ...     26    12    14   16   20    15   16     9     2   17  
46340    54  ...     23     6     9    7   24     7    9     4     4   19  
46341    46  ...     22     5    10   13   25    11   18     9     0   24  

[5 rows x 34 columns]

In [37]:
allSeasons = dfSeasonResultsDet["Season"].unique()
selectSeasons = allSeasons[np.where(allSeasons >= 2010)]
selectSeasons

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [56]:
# %%timeit -r 1 -n 1
# prepare dfStats aka potentialfeatures
def getStats(dfMatches, dfTeamNames, seasons):
    
    
    def getWinRatio(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") # in case data is not clean, using max gives the col with the most values for each team (aka the most times that team shows up as a WTeamID)
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses") # len(serWins) != len(serLosses)
        
        # len(dfTeamNames) != len(dfMatches.groupyby("wTeamID").count()); need to join by TeamID
        dfWinRatio = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWins, serLosses])
        dfWinRatio["WinRatio"] = dfWinRatio["Wins"] / (dfWinRatio["Wins"] + dfWinRatio["Losses"])
        
        return dfWinRatio
    
    
    def getPpg(dfMatches, dfTeamNames):
                
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") # in case data is not clean, using max gives the col with the most values for each team (aka the most times that team shows up as a WTeamID)
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
        
        serWinPpg = dfMatches.groupby("WTeamID").mean()["WScore"].rename_axis("TeamID").rename("WMeanScore") 
        serLosePpg = dfMatches.groupby("LTeamID").mean()["LScore"].rename_axis("TeamID").rename("LMeanScore") 

        dfPpg = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinPpg, serWins, serLosePpg, serLosses])
        dfPpg["PPG"] = ((dfPpg["WMeanScore"] * dfPpg["Wins"]) + (dfPpg["LMeanScore"] * dfPpg["Losses"])) / (dfPpg["Wins"] + dfPpg["Losses"])
        
        return dfPpg
    
    
    def getAllowedPpg(dfMatches, dfTeamNames):
                
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") # in case data is not clean, using max gives the col with the most values for each team (aka the most times that team shows up as a WTeamID)
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
        
        serWinAllowedPpg = dfMatches.groupby("WTeamID").mean()["LScore"].rename_axis("TeamID").rename("WMeanScoreAllowed") 
        serLoseAllowedPpg = dfMatches.groupby("LTeamID").mean()["WScore"].rename_axis("TeamID").rename("LMeanScoreAllowed") 
        
        dfAllowedPpg = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinAllowedPpg, serWins, serLoseAllowedPpg, serLosses])
        dfAllowedPpg["AllowedPPG"] = ((dfAllowedPpg["WMeanScoreAllowed"] * dfAllowedPpg["Wins"]) + (dfAllowedPpg["LMeanScoreAllowed"] * dfAllowedPpg["Losses"])) / (dfAllowedPpg["Wins"] + dfAllowedPpg["Losses"])
        
        return dfAllowedPpg
    
    
    def getFgPer(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
        
        serWinFgMade = dfMatches.groupby("WTeamID").mean()["WFGM"].rename_axis("TeamID").rename("WMeanFGMade") 
        serLoseFgMade = dfMatches.groupby("LTeamID").mean()["LFGM"].rename_axis("TeamID").rename("LMeanFGMade") 
        
        serWinFgAtt = dfMatches.groupby("WTeamID").mean()["WFGA"].rename_axis("TeamID").rename("WMeanFGAttempted") 
        serLoseFgAtt = dfMatches.groupby("LTeamID").mean()["LFGA"].rename_axis("TeamID").rename("LMeanFGAttempted") 

        dfFgPer = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinFgMade, serWinFgAtt, serWins, serLoseFgMade, serLoseFgAtt, serLosses])
        dfFgPer["WFGPer"] = dfFgPer["WMeanFGMade"] / dfFgPer["WMeanFGAttempted"]
        dfFgPer["LFGPer"] = dfFgPer["LMeanFGMade"] / dfFgPer["LMeanFGAttempted"]
        
        dfFgPer["FGPer"] = ((dfFgPer["WFGPer"] * dfFgPer["Wins"]) + (dfFgPer["LFGPer"] * dfFgPer["Losses"])) / (dfFgPer["Wins"] + dfFgPer["Losses"])
        
        return dfFgPer
    
    
    def getTurnovers(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
             
        serWinTurnovers = dfMatches.groupby("WTeamID").mean()["WTO"].rename_axis("TeamID").rename("WMeanTurnovers") 
        serLoseTurnovers = dfMatches.groupby("LTeamID").mean()["LTO"].rename_axis("TeamID").rename("LMeanTurnovers") 
        
        dfTurnovers = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinTurnovers, serWins, serLoseTurnovers, serLosses])
        
        dfTurnovers["Turnovers"] = ((dfTurnovers["WMeanTurnovers"] * dfTurnovers["Wins"]) + (dfTurnovers["LMeanTurnovers"] * dfTurnovers["Losses"])) / (dfTurnovers["Wins"] + dfTurnovers["Losses"])

        return dfTurnovers
    
    
    # blocks do not count as turnovers and therefore will be considered as a separate independant metric
    def getBlocks(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
               
        serWinBlocks = dfMatches.groupby("WTeamID").mean()["WBlk"].rename_axis("TeamID").rename("WMeanBlocks") 
        serLoseBlocks = dfMatches.groupby("LTeamID").mean()["LBlk"].rename_axis("TeamID").rename("LMeanBlocks") 
        
        dfBlocks = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinBlocks, serWins, serLoseBlocks, serLosses])
        dfBlocks["Blocks"] = ((dfBlocks["WMeanBlocks"] * dfBlocks["Wins"]) + (dfBlocks["LMeanBlocks"] * dfBlocks["Losses"])) / (dfBlocks["Wins"] + dfBlocks["Losses"])

        return dfBlocks
    
    
    def getRebounds(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
        
        serWinOffRebounds = dfMatches.groupby("WTeamID").mean()["WOR"].rename_axis("TeamID").rename("WMeanOffRebounds") 
        serLoseOffRebounds = dfMatches.groupby("LTeamID").mean()["LOR"].rename_axis("TeamID").rename("LMeanOffRebounds") 
        serWinDefRebounds = dfMatches.groupby("WTeamID").mean()["WDR"].rename_axis("TeamID").rename("WMeanDefRebounds") 
        serLoseDefRebounds = dfMatches.groupby("LTeamID").mean()["LDR"].rename_axis("TeamID").rename("LMeanDefRebounds") 
        
        dfRebounds = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinOffRebounds, serWinDefRebounds, serWins, serLoseOffRebounds, serLoseDefRebounds, serLosses])
        dfRebounds["WMeanRebounds"] = dfRebounds["WMeanOffRebounds"] + dfRebounds["WMeanDefRebounds"]
        dfRebounds["LMeanRebounds"] = dfRebounds["LMeanOffRebounds"] + dfRebounds["LMeanDefRebounds"]
        dfRebounds["Rebounds"] = ((dfRebounds["WMeanRebounds"] * dfRebounds["Wins"]) + (dfRebounds["LMeanRebounds"] * dfRebounds["Losses"])) / (dfRebounds["Wins"] + dfRebounds["Losses"])

        return dfRebounds
    
    
    def getAssists(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
               
        serWinAssists = dfMatches.groupby("WTeamID").mean()["WAst"].rename_axis("TeamID").rename("WMeanAssists") 
        serLoseAssists = dfMatches.groupby("LTeamID").mean()["LAst"].rename_axis("TeamID").rename("LMeanAssists") 
        
        dfAssists = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinAssists, serWins, serLoseAssists, serLosses])
        dfAssists["Assists"] = ((dfAssists["WMeanAssists"] * dfAssists["Wins"]) + (dfAssists["LMeanAssists"] * dfAssists["Losses"])) / (dfAssists["Wins"] + dfAssists["Losses"])

        return dfAssists
    
    
    # while some fouls can be are in dependance with the turnover metric, some are not; therefore, since not directly correlated, will be considered its own metric
    def getPerFouls(dfMatches, dfTeamNames):
        
        serWins = dfMatches.groupby("WTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Wins") 
        serLosses = dfMatches.groupby("LTeamID").count().max(axis = 1).rename_axis("TeamID").rename("Losses")
               
        serWinPerFouls = dfMatches.groupby("WTeamID").mean()["WPF"].rename_axis("TeamID").rename("WMeanPerFouls") 
        serLosePerFouls = dfMatches.groupby("LTeamID").mean()["LPF"].rename_axis("TeamID").rename("LMeanPerFouls") 
        
        dfPerFouls = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), [dfTeamNames[["TeamID"]], serWinPerFouls, serWins, serLosePerFouls, serLosses])
        dfPerFouls["PerFouls"] = ((dfPerFouls["WMeanPerFouls"] * dfPerFouls["Wins"]) + (dfPerFouls["LMeanPerFouls"] * dfPerFouls["Losses"])) / (dfPerFouls["Wins"] + dfPerFouls["Losses"])

        return dfPerFouls


    def getConf(df):
    # maybe need to scrape 
    # possible source https://www.ncaa.com/standings/basketball-men/d1
        return dfConf
    
    # SOS
    # alternative to SOS is RPI; however, that incorporates SOS
    # SOS takes into account WP, OWP, & OOWP; there is no need for me to include this metric right now since the only part of this metric not included in the model currently is OOWP; adding this SOS as a metric will add redundancy to the model since the metric is based off of other metrics already included in the model
    # def sos(dfMatches, dfTeamNames): return
    
    # H/A/N
    # homeAwayNeutral can only be determined when the diffVec is made 
    # def homeAwayNeutral(df): return
    
    # 3PtFG%
    # this metric is already included in FG%; by adding this metric it will include redundancy to the model; an alternative to this is use 2PtFG% & 3PtFG% separtely instead of including a metric with combines the two
    # def fgPer3Pt(df): return
    
    
    dfStats = pd.DataFrame([])
    for season in seasons:
        
        dfStatsOneSeason = dfTeamNames[["TeamID"]].copy()
        dfStatsOneSeason["Season"] = [season] * len(dfStatsOneSeason)
                
        dfWinRatio = getWinRatio(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfPpg = getPpg(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfAllowedPpg = getAllowedPpg(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfFgPer = getFgPer(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfTurnovers = getTurnovers(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfBlocks = getBlocks(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfRebounds = getRebounds(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfAssists = getAssists(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        dfPerFouls = getPerFouls(dfMatches.loc[dfMatches.Season == season], dfTeamNames)
        
        statFrames = [dfStatsOneSeason, 
                      dfWinRatio[["TeamID", "WinRatio"]],
                      dfPpg[["TeamID", "PPG"]],
                      dfAllowedPpg[["TeamID", "AllowedPPG"]],
                      dfFgPer[["TeamID", "FGPer"]],
                      dfTurnovers[["TeamID", "Turnovers"]],
                      dfBlocks[["TeamID", "Blocks"]],
                      dfRebounds[["TeamID", "Rebounds"]],
                      dfAssists[["TeamID", "Assists"]],
                      dfPerFouls[["TeamID", "PerFouls"]]]
        dfStatsOneSeason = reduce(lambda left, right: pd.merge(left, right, how = "outer", on = "TeamID"), statFrames)
        dfStats = pd.concat([dfStats, dfStatsOneSeason], axis = 0, ignore_index = True)
        
    dfStats = dfStats.iloc[2:-1].round(3)
        
    return dfStats
    
dfStats = getStats(dfSeasonResultsDet, dfTeamNames, selectSeasons)
dfStats.to_csv("../data/interim/TeamStats.csv", index = None, header = True)
dfStats.tail()

TeamID  Season  WinRatio     PPG  AllowedPPG  FGPer  Turnovers  Blocks  \
3288    3461    2018     0.655  60.828      57.759  0.416     13.172   2.172   
3289    3462    2018     0.333  58.633      66.067  0.396     15.667   1.833   
3290    3463    2018     0.536  67.214      65.357  0.398     15.036   2.679   
3291    3464    2018     0.483  62.414      65.069  0.399     14.586   3.172   
3292    3465    2018       NaN     NaN         NaN    NaN        NaN     NaN   

      Rebounds  Assists  PerFouls  
3288    33.690   13.586    13.621  
3289    32.867   12.133    14.033  
3290    37.357   12.643    16.679  
3291    32.931   13.414    16.724  
3292       NaN      NaN       NaN